In [ ]:
import pandas as pd
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

In [ ]:
# Datenquellen laden
data2012 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2012_edit.txt", sep=",!,", engine="python")
data2013 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2013_edit.txt", sep=",!,", engine="python")
data2014 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2014_edit.txt", sep=",!,", engine="python")
data2015 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2015_edit.txt", sep=",!,", engine="python")
data2016 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2016_edit.txt", sep=",!,", engine="python")
data2017 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2017_edit.txt", sep=",!,", engine="python")
data2018 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2018_edit.txt", sep=",!,", engine="python")
data2019 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2019_edit.txt", sep=",!,", engine="python")
data2020 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2020_edit.txt", sep=",!,", engine="python")
data2021 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2021_edit.txt", sep=",!,", engine="python")
data2022 = pd.read_csv(r"C:\Beispielnutzer\sdm_meldungen_2022_edit.txt", sep=",!,", engine="python")

In [ ]:
# Dataframes joinen
frames = [data2012, data2013, data2014, data2015, data2016, data2017, data2018, data2019, data2020, data2021, data2022] 
data = pd.concat(frames)

In [ ]:
# Relevante Datenpunkte auswählen
data = data[(data.status == "Geschlossen") | (data.status == "Gelöst")] 

In [ ]:
# NaN eliminieren
data = data.where(pd.notnull(data)," ")

In [ ]:
# Format von float zu string ändern
data['betroffener_endbenutzer_organisation'] = data['betroffener_endbenutzer_organisation'].astype('str') 
data['beschreibung'] = data['beschreibung'].astype('str')
data['zusammenfassung'] = data['zusammenfassung'].astype('str')
data['configuration_items'] = data['configuration_items'].astype('str')

In [ ]:
#Text Vorverarbeitung
def preprocess(text):
    text = text.lower() # Der Text wird in kleine Buchstaben umgewandelt
    text = text.strip()  # Führende/nachlaufende Leerzeichen entfernen
    text = re.compile('<.*?>').sub('', text) #Entfernt HTML tags/markups
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  #Ersetzt Sie Satzzeichen durch Leerzeichen
    text = re.sub('\s+', ' ', text)  #Entfernt zusätzliche Leerzeichen und Tabulatoren 
    text = re.sub(r'\[[0-9]*\]',' ',text) #[0-9] Entspricht einer beliebigen Ziffer (0 bis 10000...)
    text = re.sub(r'\d',' ',text) #Passt auf jede Ziffer von 0 bis 100000...,
    text = re.sub(r'\s+',' ',text) #\s entspricht einem beliebigen Leerzeichen, \s+ entspricht mehreren Leerzeichen, \S entspricht Nicht-Leerzeichen 
    return text

In [ ]:
# Stopwords entfernen
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('german')]
    return ' '.join(a)

In [ ]:
# Funktionen stopwords und preprocess anwenden
data['zusammenfassung'] = data['zusammenfassung'].apply(lambda x: stopword(preprocess(x)))
data['beschreibung'] = data['beschreibung'].apply(lambda x: stopword(preprocess(x)))

In [ ]:
# Lemmatisierungsfunktion definieren
def lemmaDeu(df : object, column : str):
    nlp = spacy.load('de_core_news_md')
    df.loc[:, column] = df[column].apply(lambda x: ' '.join([a.lemma_ for a in nlp(x)]))
    return df

In [ ]:
# Lemmatisierung anwenden
lemmaDeu(data,"beschreibung")
lemmaDeu(data,"zusammenfassung")

In [ ]:
# Nicht weiter genutzte Spalten entfernen
data = data.drop(["id", "ticket_id", "erstellt_am","status","gruppe_name", "service_solution"], 1)

In [ ]:
#Gewichtung der Spalten verändern + Zusammenfassen der Spalten
x_col = ["beschreibung","meldungseingang","betroffener_endbenutzer_organisation","configuration_items","zusammenfassung"]
output = ["service_solution_id","gruppe_id"]

In [ ]:
# Funktion zum ändern der Gwichtung
def weightModification(df : object, factor : int, x_col : list):
        df.loc[:, "zusammenfassung"] = df["zusammenfassung"].apply(lambda x: (x+" ")*factor)
        df.loc[:, "configuration_items"] = df["configuration_items"].apply(lambda x: (x+" ")*factor)
        df["input"] = df[x_col].apply(" ".join, axis="columns")
        df = df.drop(x_col,1)
        df["input"] = df["input"].str.lower()
        return df

In [ ]:
# Gewichtungsfunktion anwenden
data = weightModification(data,5,x_col)

In [ ]:
# Funktion Tokenisierung definieren
def tokenize(df : object, column : str):
    df[column] = df.apply(lambda row: word_tokenize(row[column]), axis=1)
    return df

In [ ]:
#Funktion Tokenisierung anwenden
tokenize(data,"input")

In [ ]:
#Dataframe wird als .csv Datei gespeichert
data.to_csv('sdm_meldungen_processed.csv', sep=',')